In [1]:
import sys
sys.path.append('/home/ec2-user/SageMaker/mariano/repositories/tdmstudio-high-recall-information-retrieval-system/')
from utils.data_item import DataItem
from utils.oracle import Oracle
# unlabeled = Oracle.get_collection()

from utils.scal_ui import SCAL_UI
import os
from IPython.display import clear_output
from utils.scal import SCAL
from utils.data_item import QueryDataItem

# 


seed = 22127
N=30000
cap=5
def start_system(session_name, topic_description ):
    clear_output(wait=False)
    if os.path.exists(f'sessions/scal/{session_name}'):
        scal = SCAL.from_disk(session_name=session_name)
    else:
        
        # UNLABELED
#         data_sources = ['/home/ec2-user/SageMaker/mariano/notebooks/07. Simulation/data/GM_files'
#                        ]
        data_sources = ['/home/ec2-user/SageMaker/data/GM_not_all_1960_1978',
                        '/home/ec2-user/SageMaker/data/GM_not_all_1979_1997',
                        '/home/ec2-user/SageMaker/data/GM_not_all_1998_2018',
                    ]
        files = [os.path.join(data_source,file) for data_source in data_sources for file in os.listdir(data_source) ]
#         print(f'Total number of files: {len(files):,}')

        
        def unlabeled_from_data_source(data_source):
            return [DataItem(name.split('.')[0]) for name in os.listdir(data_source)]
        
        unlabeled = [elem for list_ in list(map(unlabeled_from_data_source, data_sources)) for elem in list_]
#         unlabeled = list(filter(lambda x: x.has_vector(), unlabeled))
        # LABELED
#         print(topic_description)
        doc = QueryDataItem(topic_description)
        doc.set_relevant()
        scal = SCAL(session_name, 
                    [doc], 
                    unlabeled,
                    random_sample_size=N,
                    batch_size_cap=cap,
                    simulation=False, 
                    seed=seed)
    scal.run()
    
_=SCAL_UI(start_system)

# import ipywidgets as widgets
# combobox = widgets.Combobox(options=['1','2','3',])
# def do_something(widget=None):
#     print('submit')
# combobox.on_submit(do_something)
# combobox

HBox()

In [16]:
from utils.tokenizer import Tokenizer
tokenizer = Tokenizer()
# Tokenizer.ngrams(list(tokenizer.tokenize()))

token_list = tokenizer.tokenize('New York Times')   # With lower case lemmatization works different and 
                                                    # New York Times trigrams is missed.
ngram_list = list(token_list)
ngram_list += [' '.join(ngram) for ngram in Tokenizer.ngrams(ngram_list)]
ngram_list = filter(lambda ngram: ngram in QueryDataItem.word2index , ngram_list)
list(ngram_list)

['new', 'york', 'times', 'new york', 'york times', 'new york times']

In [3]:
import numpy as np
from utils.data_item import QueryDataItem
vocab = np.array(QueryDataItem.vocab)
print(f'len(vocab)={len(vocab)}')

syn=QueryDataItem('new york times')
print(f'Topic description: {syn.text}')
print(f'Vector norm:       {np.linalg.norm(syn.vector()):4.3f}')
print(f'N-grams:           {vocab[syn.vector()!=0]}')
print(f'Weights:           {syn.vector()[syn.vector()!=0]}')


len(vocab)=10000
Topic description: new york times 
Vector norm:       1.000
N-grams:           ['new' 'time' 'york' 'new york']
Weights:           [0.24018316 0.30329878 0.55767806 0.73438211]


In [21]:
data_sources = ['/home/ec2-user/SageMaker/data/GM_not_all_1960_1978',
                '/home/ec2-user/SageMaker/data/GM_not_all_1979_1997',
                '/home/ec2-user/SageMaker/data/GM_not_all_1998_2018',
            ]
files = [os.path.join(data_source,file) for data_source in data_sources for file in os.listdir(data_source) ]
print(f'len(files)={len(files)}')
#         print(f'Total number of files: {len(files):,}')


def unlabeled_from_data_source(data_source):
    return [DataItem(name.split('.')[0]) for name in os.listdir(data_source)]

unlabeled = [elem for list_ in list(map(unlabeled_from_data_source, data_sources)) for elem in list_]


len(files)=2961906


In [22]:
suggestion =[item for item in unlabeled if item.id_=='1136514515'][0]

In [20]:
len(unlabeled)

7280

array(['canada', 'year', 'new', ..., 'toronto city', 'multicultural',
       'multiculturalism'], dtype='<U30')

In [11]:
vocab[syn.vector()!=0]

array(['car', 'vehicle', 'automobile'], dtype='<U30')

In [1]:
from utils.data_item import QueryDataItem
doc = QueryDataItem("multiculturalism multicultural")
doc.set_relevant()
doc.vector()

array([0.        , 0.        , 0.        , ..., 0.        , 0.64985456,
       0.76005859])

In [ ]:
array([0.        , 0.        , 0.        , ..., 0.        , 0.70710678,
       0.70710678])

In [2]:
data_sources = ['/home/ec2-user/SageMaker/mariano/notebooks/07. Simulation/data/GM_files'
               ]
def unlabeled_from_data_source(data_source):
    return [DataItem(name.split('.')[0]) for name in os.listdir(data_source)]
unlabeled = [elem for list_ in list(map(unlabeled_from_data_source, data_sources)) for elem in list_]
unlabeled = list(filter(lambda x: x.has_vector(), unlabeled))

In [7]:
import pickle
from sklearn.preprocessing import normalize

vectors=[]
for vecname in [item._vector_filename() for item in unlabeled]:
    vectors.append(pickle.load(open(vecname,'rb'))['BoW'].toarray()[0,:])
    
X = normalize(sparse.vstack(map(lambda filename: pickle.load(open(filename, 'rb'))['BoW']), vecnames[ini:fin])),axis=1)


(10000,)

In [10]:
from scipy import sparse
vecnames = [item._vector_filename() for item in unlabeled]
sparse.vstack(map(lambda filename: pickle.load(open(filename, 'rb'))['BoW'], vecnames[0:100]))


<100x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 18397 stored elements in Compressed Sparse Row format>

In [6]:
!ls /home/ec2-user/SageMaker/mariano/notebooks/07.\ Simulation/data/precomputed | head

1124938409_representations.p
1124939367_representations.p
1124939834_representations.p
1124940161_representations.p
1124941625_representations.p
1124942915_representations.p
1124943522_representations.p
1124945002_representations.p
1124945205_representations.p
1124945670_representations.p
ls: write error: Broken pipe


In [7]:
!ls /home/ec2-user/SageMaker/mariano/notebooks/07.\ Simulation/data/GM_files | head

1124938409.xml
1124939367.xml
1124939834.xml
1124940161.xml
1124941625.xml
1124942915.xml
1124943522.xml
1124945002.xml
1124945205.xml
1124945670.xml
ls: write error: Broken pipe


In [10]:
xmlfiles = ['data/GM_files/'+file for file in os.listdir('data/GM_files')]
assert all([os.path.isfile(file) for file in xmlfiles])
len(xmlfiles)

30000

In [11]:
vecfiles = ['data/precomputed/'+file for file in os.listdir('data/precomputed/')]
assert all([os.path.isfile(file) for file in vecfiles])
len(vecfiles)

30000